### Bayesian Optimization on DynamicExpressions


Import necessary packages

In [19]:
import juliapkg
import juliacall
import numpy as np

Install Julia packages

In [20]:
try:
    from juliacall import Main as jl
except:
    juliapkg.require_julia("=1.10.4")
    juliapkg.add(
        "Optim",
        "429524aa-4258-5aef-a3af-852621145aeb",
        version="=1.9.4",
    )
    juliapkg.add(
        "DynamicExpressions",
        "a40a106e-89c9-4ca8-8020-a735e8728b6b",
        version="=0.18.5",
    )
    juliapkg.resolve()

from juliacall import Main as jl

Load main expression package

In [21]:
%%julia
using DynamicExpressions

> **NOTE:**
>
> Here, we used the `%%julia` magic command. Normally you would wrap this with a `jl.seval(""" ... """)` call.

Load dependency for constant optimization

In [22]:
%%julia
using Optim: Optim

Define a function that creates a single node from the node type and payload

In [45]:
%%julia

function create_node((node_type, payload), operators::OperatorEnum)
    if node_type == "constant"
        return Node{Float64}(; val=one(Float64))
    elseif node_type == "feature"
        return Node{Float64}(; feature=payload)
    else
        n = Node{Float64}()
        degree = node_type == "unary" ? 1 : 2
        n.degree = degree
        if degree == 1
            @assert(
                payload in eachindex(operators.unaops),
                "Could not find unary operator with index $payload"
            )
        else
            @assert(
                payload in eachindex(operators.binops),
                "Could not find binary operator with index $payload"
            )
        end
        n.op = payload
        return n
    end
end


create_node (generic function with 1 method)

Define a function that generates the full tree from a list of arbitrary nodes.
This should work even if the list of nodes is invalid.

In [46]:
%%julia

function create_tree(nodes::AbstractVector{Tuple{String,Int}}, operators::OperatorEnum)::Node
    _create_tree!(copy(nodes), operators)
end

function _create_tree!(nodes::AbstractVector{Tuple{String,Int}}, operators::OperatorEnum)::Node
    if isempty(nodes)
        # Fallback is to simply return 1.0
        return create_node(("constant", 0), operators)
    end
    new_root = create_node(popfirst!(nodes), operators)
    if new_root.degree == 0
        # Nothing to do
    elseif new_root.degree == 1
        new_root.l = _create_tree!(nodes, operators)
    elseif new_root.degree == 2
        new_root.l = _create_tree!(nodes, operators)
        new_root.r = _create_tree!(nodes, operators)
    else
        error()
    end
    return new_root
end


_create_tree! (generic function with 2 methods)

Define a function that returns the loss for a given tree

In [68]:
%%julia

function loss(operators::OperatorEnum, X::Matrix{Float64}, y::Vector{Float64})
    let X_t = X'
        function (tree)
            y_predicted = first(eval_tree_array(tree, X_t, operators))
            return sum(i -> abs2(y[i] - y_predicted[i]), eachindex(y, y_predicted))
        end
    end
end


loss (generic function with 2 methods)

Define a function that wraps `create_tree` with `loss`,
so we can just pass in our list of nodes, dataset, operators,
and get a result back.

In [69]:
%%julia

function evaluate(nodes::AbstractVector{Tuple{String,Int}}, operators::OperatorEnum, X::AbstractMatrix{Float64}, y::AbstractVector{Float64})
    tree = create_tree(nodes, operators)
    l = loss(operators, X, y)
    if !has_constants(tree)
        # Nothing to optimize
        return (l(tree), tree)
    end
    res = Optim.optimize(l, tree, Optim.BFGS())
    return (res.minimum, res.minimizer)
end


evaluate (generic function with 3 methods)

Define the space of possibilities we can sample from

In [70]:
%%julia

function possibilities(n_features, operators::OperatorEnum)
    possible_nodes = [
        ("binary", i) for i in eachindex(operators.binops)
    ]
    append!(possible_nodes, [
        ("unary", j) for j in eachindex(operators.unaops)
    ])
    append!(possible_nodes, [
        ("feature", k) for k in 1:n_features
    ])
    append!(possible_nodes, [("constant", 0)])
    return possible_nodes
end


possibilities (generic function with 1 method)

Now, let's get this set in Python:


In [71]:
operators = jl.seval("OperatorEnum(binary_operators=(+, -, *), unary_operators=(sin, exp))")
num_features = 3
jl.possibilities(num_features, operators)

9-element Vector{Tuple{String, Int64}}:
 ("binary", 1)
 ("binary", 2)
 ("binary", 3)
 ("unary", 1)
 ("unary", 2)
 ("feature", 1)
 ("feature", 2)
 ("feature", 3)
 ("constant", 0)

This is a Julia array. But we can convert it to a numpy array too:

In [72]:
np.array(jl.possibilities(num_features, operators))

array([('binary', 1), ('binary', 2), ('binary', 3), ('unary', 1),
       ('unary', 2), ('feature', 1), ('feature', 2), ('feature', 3),
       ('constant', 0)], dtype=object)

Which means we can sample this!

In [73]:
possibilities = np.array(jl.possibilities(num_features, operators))
max_nodes = 30
nodes = np.random.choice(possibilities, max_nodes)

nodes

array([('feature', 3), ('unary', 2), ('constant', 0), ('unary', 2),
       ('binary', 1), ('feature', 1), ('unary', 1), ('feature', 2),
       ('binary', 2), ('constant', 0), ('unary', 1), ('feature', 1),
       ('feature', 3), ('unary', 1), ('feature', 3), ('feature', 2),
       ('binary', 1), ('binary', 3), ('binary', 1), ('feature', 2),
       ('unary', 1), ('feature', 1), ('unary', 1), ('constant', 0),
       ('binary', 1), ('feature', 2), ('binary', 3), ('feature', 2),
       ('feature', 2), ('binary', 3)], dtype=object)

Now, let's convert this into a tree and evaluate it. Re-run this cell to get new results:


In [138]:

# Data:
X = np.random.randn(100, num_features)
# True relation we wish to predict:
y = np.cos(X[:, 0] * 2.1 - 0.5) + X[:, 1]

# Sample possibilities:
nodes = np.random.choice(possibilities, max_nodes)

# Evaluate, via also optimizing the constants in the tree:
(loss, tree) = jl.evaluate(
    juliacall.convert(jl.seval("Vector{Tuple{String, Int}}"), nodes),
    operators,
    juliacall.convert(jl.Matrix, X),
    juliacall.convert(jl.Vector, y)
)
loss, tree

(120.63556100144872,
 Julia: sin(sin((sin(-1.5071308401669326) - (exp(x2 + -43.92876236124431) + ((x1 - x2) - exp(sin((exp(26.456843581641678) + x1) + (x2 + x2)))))) + exp(sin(x2 - x2)))))